In [1]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

import os
import re
import numpy as np
import pandas as pd
import collections

In [2]:
x_name = os.path.join(os.getcwd(), '데이터SET', f"{'[Track1_데이터3] samp_cst_feat'}.csv")
df_x = pd.read_csv(x_name, index_col=0)

y_name = os.path.join(os.getcwd(), '데이터SET', f"{'[Track1_데이터2] samp_train'}.csv")
df_y = pd.read_csv(y_name, index_col=0)

df = pd.merge(df_x, df_y, on='cst_id_di')

In [3]:
df.MRC_ID_DI[df.MRC_ID_DI > 0 ] = 1

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [4]:
nc_name = os.path.join(os.getcwd(), '데이터SET', f"{'[Track1_데이터4] variable_dtype'}.xlsx")
nc = pd.read_excel(nc_name, index_col=0)

In [5]:
df = df.drop(columns = ['MRC_ID_DI', 'VAR021', 'VAR046'], axis=1)

In [6]:
df[df.columns] = (df[df.columns]+1)/2*255

In [ ]:
from PIL import Image
import numpy as np
for ind, k in enumerate(df.index):
    data = np.zeros((224, 224, 3), dtype=np.uint8)
    for i in range(224):
        for j in range(224):
            data[i, j] = [(df.iloc[ind, j] + df.iloc[ind, i])//2, (df.iloc[ind, j] + df.iloc[ind, i])//2, (df.iloc[ind, j] + df.iloc[ind, i])//2]
    img = Image.fromarray(data, 'RGB')
    img_name = os.path.join(os.getcwd(), 'image2', str(k) + '.png')
    img.save(img_name)